In [3]:
import pandas as pd

In [5]:
df1 = pd.read_csv("Покупки сотрудников в боброшопе (1).csv", delimiter=';')
df2 = pd.read_excel("Список сотрудников КРОК (1).xlsx")
df3 = pd.read_csv("NEW Как зарабатывают бобров КРОК (1).csv", delimiter=';')

ОБЪЕДИНЯЕМ ТАБЛИЦЫ СО СПИСКОМ СОТРУДНИКОВ И ЗАРАБОТКОМ

In [6]:
df2 = df2.rename(columns={'Внешний код': 'Код сотрудника'})
merged_df = pd.merge(df3, df2, on='Код сотрудника', how='left')
merged_df.drop(columns=['Комментарий'], inplace=True)

In [7]:
merged_df.head()

,Код сотрудника,Департамент,Сумма вознаграждения,Название миссии,Контейнер,Дата,Факт. департамент,Факт. подразделение,Факт. группа,Факт. должность,Является РМ/ТЛ,Стаж фактический по компании,Пол,Возраст
0,SU147663,ДБП.группа консультантов по автоматизации бизн...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,06.02.2024,Департамент бизнес-приложений,NaN,Группа консультантов по автоматизации бизнес-п...,Консультант по внедрению бизнес-приложений,нет,"4 г., 2 мес.,28 дн.",Женский,28.0
1,SU76490,ДВА.группа автоматизации производственных проц...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,24.01.2024,Департамент внутренней автоматизации,NaN,группа автоматизации производственных процессов,Ведущий системный аналитик,нет,"9 г., 7 мес.,18 дн.",Женский,31.0
2,SU69215,ДИРС.группа инженеров по ИТ-инфраструктуре и м...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,29.07.2024,Департамент инфраструктурных решений и сервисов,группа инженеров по ИТ-инфраструктуре и монито...,NaN,Системный инженер,нет,"11 г., 0 мес.,23 дн.",Мужской,33.0
3,SU4528,Служба охраны.группа обеспечения контрольно-пр...,5.0,Ты угадал коллегу в Jive с первого раза,Корпоративная жизнь КРОК,02.05.2024,Служба охраны,NaN,Группа обеспечения контрольно-пропускного режима,Охранник,нет,"15 г., 5 мес.,24 дн.",Мужской,37.0
4,SU163517,ДРП.отдел кадров,5.0,Ты угадал коллегу в Jive с первого раза,Корпоративная жизнь КРОК,06.09.2024,Департамент по работе с персоналом,отдел кадров,NaN,HR-менеджер,нет,"3 г., 4 мес.,23 дн.",Женский,27.0


СМОТРИМ, КАКИЕ ЕСТЬ МИССИИ И ЧЕРЕЗ СРЕДНЕЕ УЗНАЕМ ИХ ВОЗНАГРАЖДЕНИЕ

In [21]:
mission_rewards = merged_df.groupby('Название миссии')['Сумма вознаграждения'].agg(['mean', 'count']).reset_index()
print("Сумма и среднее вознаграждение за каждую миссию:")
mission_rewards

Сумма и среднее вознаграждение за каждую миссию:


,Название миссии,mean,count
0,Best Hunter,600.0,112
1,Best Hunter (Вакансия месяца),800.0,6
2,CROC Energy,400.0,3
3,CROC Voices,500.0,61
4,Croc Sport,NaN,0
...,...,...,...
89,Ты стал экспертом/членом жюри на кейс-чемпиона...,250.0,7
90,Ты угадал коллегу в Jive с первого раза,5.0,30419
91,Ты успешно сдал экзамен*,150.0,7
92,Участник Manager's fight,200.0,40


УДАЛЯЕМ ПУСТЫЕ

In [9]:
mission_rewards.dropna()

,mean,count
Название миссии,,
Best Hunter,600.0,112
Best Hunter (Вакансия месяца),800.0,6
CROC Energy,400.0,3
CROC Voices,500.0,61
Guardian of CROC,150.0,1
...,...,...
"Ты стал экспертом/членом жюри на кейс-чемпионате, хакатоне, конкурсе, Demo акселератора",250.0,7
Ты угадал коллегу в Jive с первого раза,5.0,30419
Ты успешно сдал экзамен*,150.0,7


А теперь начинаются танцы с бубном вокруг миссий.


---


Делим их следующим образом:

если их цена от 1 до 30 - миссия не важна вообще

от 40 до 250 - полезная

300 и более - очень полезная

In [13]:
bins = [0, 30, 250, float('inf')]
labels = ["Миссия не важна", "Полезная", "Очень полезная"]
merged_df['Важность миссии'] = pd.cut(
    merged_df['Сумма вознаграждения'],
    bins=bins,
    labels=labels,
    right=True
)

In [11]:
merged_df.head()

,Код сотрудника,Департамент,Сумма вознаграждения,Название миссии,Контейнер,Дата,Факт. департамент,Факт. подразделение,Факт. группа,Факт. должность,Является РМ/ТЛ,Стаж фактический по компании,Пол,Возраст,Важность миссии
0,SU147663,ДБП.группа консультантов по автоматизации бизн...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,06.02.2024,Департамент бизнес-приложений,NaN,Группа консультантов по автоматизации бизнес-п...,Консультант по внедрению бизнес-приложений,нет,"4 г., 2 мес.,28 дн.",Женский,28.0,Полезная
1,SU76490,ДВА.группа автоматизации производственных проц...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,24.01.2024,Департамент внутренней автоматизации,NaN,группа автоматизации производственных процессов,Ведущий системный аналитик,нет,"9 г., 7 мес.,18 дн.",Женский,31.0,Полезная
2,SU69215,ДИРС.группа инженеров по ИТ-инфраструктуре и м...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,29.07.2024,Департамент инфраструктурных решений и сервисов,группа инженеров по ИТ-инфраструктуре и монито...,NaN,Системный инженер,нет,"11 г., 0 мес.,23 дн.",Мужской,33.0,Полезная
3,SU4528,Служба охраны.группа обеспечения контрольно-пр...,5.0,Ты угадал коллегу в Jive с первого раза,Корпоративная жизнь КРОК,02.05.2024,Служба охраны,NaN,Группа обеспечения контрольно-пропускного режима,Охранник,нет,"15 г., 5 мес.,24 дн.",Мужской,37.0,Миссия не важна вообще
4,SU163517,ДРП.отдел кадров,5.0,Ты угадал коллегу в Jive с первого раза,Корпоративная жизнь КРОК,06.09.2024,Департамент по работе с персоналом,отдел кадров,NaN,HR-менеджер,нет,"3 г., 4 мес.,23 дн.",Женский,27.0,Миссия не важна вообще


Создаем сводную таблицу по количеству задач каждого типа для каждого сотрудника

In [29]:
task_counts = merged_df.pivot_table(
    index='Код сотрудника',
    columns='Важность миссии',
    values='Название миссии',
    aggfunc='count',
    fill_value=0
).add_prefix('Количество ')

key_missions = ['Креативный класс', 'HiPo', 'HiPro', 'Звезда департамента']
for mission in key_missions:
    merged_df[mission] = merged_df['Название миссии'].str.contains(mission, na=False)

key_mission_stats = merged_df.groupby('Код сотрудника')[key_missions].any()

employee_stats = pd.concat([task_counts, key_mission_stats], axis=1)

employee_info = merged_df.drop_duplicates('Код сотрудника').set_index('Код сотрудника')
employee_stats = employee_stats.join(employee_info[['Факт. департамент', 'Факт. должность']])

employee_stats.reset_index(inplace=True)

employee_stats.columns = [
    'Код сотрудника',
    'Не важные миссии',
    'Полезные миссии',
    'Очень полезные миссии',
    'Выполнил Креативный класс',
    'Выполнил HiPo',
    'Выполнил HiPro',
    'Выполнил Звезда департамента',
    'Департамент',
    'Должность'
]

employee_stats.fillna({
    'Не важные миссии': 0,
    'Полезные миссии': 0,
    'Очень полезные миссии': 0,
    'Выполнил Креативный класс': False,
    'Выполнил HiPo': False,
    'Выполнил HiPro': False,
    'Выполнил Звезда департамента': False
}, inplace=True)
employee_stats.head()

<ipython-input-29-43dd170eed3a>:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  task_counts = merged_df.pivot_table(


,Код сотрудника,Не важные миссии,Полезные миссии,Очень полезные миссии,Выполнил Креативный класс,Выполнил HiPo,Выполнил HiPro,Выполнил Звезда департамента,Департамент,Должность
0,Bobr,0,0,1,False,False,False,False,NaN,NaN
1,SU100026,0,1,0,False,False,False,False,Департамент маркетинга,Бизнес-партнёр по маркетингу
2,SU100135,7,13,0,False,False,False,False,Департамент инженерных и мультимедийных систем,Ведущий инженер
3,SU100238,1,11,0,False,False,False,False,Департамент инфраструктурных решений и сервисов,Инженер технической поддержки
4,SU100262,1,14,4,True,False,False,False,Департамент инфраструктурных решений и сервисов,Менеджер по продвижению решений


Исходя из описания, сотрудники, имеющие 4 статуса которые я описала выше, являются эффективными для компании. Поэтому мы можем посчитать по ним статистику выполненных миссий, и из них долю полезных и очень полезных. Усредним, и получим минимальный показатель, по которому мы понимаем, что сотрудник эффективен